In [66]:
import matplotlib.pyplot as plt
from PIL import Image
from PIL.Image import NEAREST
import numpy as np
import cv2
import os
import imutils
from random import random, seed, randint

from scipy import ndimage as ndi
from scipy.spatial.distance import euclidean
from skimage.morphology import watershed, disk
from skimage.feature import peak_local_max
from torchvision.transforms.functional import pad, resized_crop, resize, center_crop, rotate, hflip, to_pil_image
# from torch.random import torch.random.manual_seed(seed)

from processing import *

In [45]:
# o=get_origs()
# get_clahes(o)
# cell_masks = get_cell_masks()
# cell_markers = get_cell_markers(cell_masks, 24)
# weight_maps = get_weight_maps(cell_markers, False)

# pimg(weight_maps[1])

def plot_two_images(imgL, imgR, titleL, titleR):
    f = plt.figure()
    f.add_subplot(1,2, 1)
    plt.imshow(imgL, cmap='gray')
    plt.title(titleL)
    f.add_subplot(1,2, 2)
    plt.imshow(imgR) #, cmap='gray')
    plt.title(titleR)
    plt.show(block=True)

In [3]:
def get_pred_masks():
    return get_binary_cell_masks(get_cell_masks())

def get_pred_markers(pred_masks, e_r):
    return get_binary_cell_markers(get_cell_masks(), e_r)

In [4]:
def threshold_binary_image(img, thresh=0.5):
    image = img.copy()
    return (image*1.0 >= thresh).astype('uint8')

In [70]:
def img_aug(dataset, new_prefix = "_aug"):
    #constants
    num_aug_multiplier = 10
    
    new_dir_name = dataset + new_prefix
    
    try:
        os.mkdir(new_dir_name)
        os.mkdir(os.path.join(new_dir_name, "originals"))
        os.mkdir(os.path.join(new_dir_name, "masks"))
    except:
        print("One of the directories to be created already exists. Clear and try again.")
        return
    
    data = []
    paths = [dataset + '/01', dataset + '/02']
    count = 0
    
    for path in paths:
        mask_path = path + '_ST'
        mask_path = os.path.join(mask_path, "SEG")
        for f in os.listdir(mask_path):
            if not f.endswith(".tif"):
                continue
            image_np = cv2.imread(os.path.join(path, f.replace('man_seg', 't')), cv2.IMREAD_GRAYSCALE)
            mask_np = cv2.imread(os.path.join(mask_path, f), cv2.IMREAD_UNCHANGED)
            
#             image_pil = Image.open( os.path.join(path, f.replace('man_seg', 't')) )
#             mask_pil = Image.open( os.path.join(mask_path, f) )
            image_pil = to_pil_image(image_np)
            mask_pil = Image.open( os.path.join(mask_path, f) )
            mask_pil = mask_pil.convert("L")

            print("   Loaded " + os.path.join(mask_path, f) + ", " + os.path.join(path, f.replace('man_seg', 't')))
            
            index = 't' + str(count) + ".tif"
            count += 1
            
            cv2.imwrite(os.path.join(new_dir_name, "originals", index ), image_np)
            cv2.imwrite(os.path.join(new_dir_name, "masks", index ), mask_np)
            
            for i in range(num_aug_multiplier):
                new_img = None
                new_mask = None
                image = image_pil
                mask = mask_pil
                
                seed()
                seed_ = randint(0,1000)
                
                index = 't' + str(count) + ".tif"
                count += 1
                
                seed(seed_)
                if random() > 0.5:
                    pad_size = int(randint(0,int(image.height*0.2)))
                    pad_size = (pad_size)
                    new_img = pad(image, pad_size, padding_mode='reflect')
                    new_img = resize(new_img, (image.height, image.width))
                else:
                    t_size = image.height * (0.6 + ( random() * 0.39 ) )
                    new_img = center_crop(image, (t_size, t_size))
                    new_img = resize(new_img, (image.height, image.width) )
                
                pad_size = int(image.height*0.5)
                pad_size = (pad_size)
                new_img = pad(new_img, pad_size, padding_mode='reflect')
                new_img = rotate(new_img, randint(0,360), expand=False, fill=0)
#                 new_img = resize(new_img, (image.height, image.width) )
                new_img = center_crop(new_img, (image.height, image.width) )
                
                new_img = hflip(new_img)
                new_img.save( os.path.join(new_dir_name, "originals", index) )
                
                seed(seed_)
                if random() > 0.5:
                    pad_size = int(randint(0, int(mask.height*0.2)))
                    pad_size = (pad_size)
                    new_mask = pad(mask, pad_size, padding_mode='reflect')
                    new_mask = resize(new_mask, (mask.height, mask.width))
                else:
                    t_size = mask.height * (0.6 + ( random() * 0.39 ) )
                    new_mask = center_crop(mask, (t_size, t_size) )
                    new_mask = resize(new_mask, (mask.height, mask.width))
                
                pad_size = int(mask.height*0.5)
                pad_size = (pad_size)
                new_mask = pad(new_mask, pad_size, padding_mode='reflect')
                new_mask = rotate(new_mask, randint(0,360), expand=False, fill=0)
#                 new_mask = resize(new_mask, (mask.height, mask.width), interpolation=NEAREST )
                new_mask = center_crop(new_mask, (mask.height, mask.width) )
                
                new_mask = hflip(new_mask)
                new_mask.save( os.path.join(new_dir_name, "masks", index ) )
                
#                 plot_two_images(new_img, new_mask, "Gen Img", "Gen Mask")
                print(index)
                
img_aug("DIC-2")
# #             np.save(os.path.join(new_dir_name, "originals", index ), image)
# #             np.save(os.path.join(new_dir_name, "clahes", index ), clahe)
# #             np.save(os.path.join(new_dir_name, "masks", index ), cell_mask)
# #             np.save(os.path.join(new_dir_name, "markers", index ), markers)
# #             np.save(os.path.join(new_dir_name, "weight_maps", index ), weight_map)
            
#             cv2.imwrite(os.path.join(new_dir_name, "originals", index ), image)
#             cv2.imwrite(os.path.join(new_dir_name, "masks", index ), cell_mask)
#             cv2.imwrite(os.path.join(new_dir_name, "markers", index ), markers)
#             cv2.imwrite(os.path.join(new_dir_name, "weight_maps", index ), weight_map)

   Loaded DIC-2/01_ST\SEG\man_seg000.tif, DIC-2/01\t000.tif
t1.tif
t2.tif
t3.tif
t4.tif
t5.tif
t6.tif
t7.tif
t8.tif
t9.tif
t10.tif
   Loaded DIC-2/01_ST\SEG\man_seg001.tif, DIC-2/01\t001.tif
t12.tif
t13.tif
t14.tif
t15.tif
t16.tif
t17.tif
t18.tif
t19.tif
t20.tif
t21.tif
   Loaded DIC-2/01_ST\SEG\man_seg002.tif, DIC-2/01\t002.tif
t23.tif
t24.tif
t25.tif
t26.tif
t27.tif
t28.tif
t29.tif
t30.tif
t31.tif
t32.tif
   Loaded DIC-2/01_ST\SEG\man_seg003.tif, DIC-2/01\t003.tif
t34.tif
t35.tif
t36.tif
t37.tif
t38.tif
t39.tif
t40.tif
t41.tif
t42.tif
t43.tif
   Loaded DIC-2/01_ST\SEG\man_seg004.tif, DIC-2/01\t004.tif
t45.tif
t46.tif
t47.tif
t48.tif
t49.tif
t50.tif
t51.tif
t52.tif
t53.tif
t54.tif
   Loaded DIC-2/01_ST\SEG\man_seg005.tif, DIC-2/01\t005.tif
t56.tif
t57.tif
t58.tif
t59.tif
t60.tif
t61.tif
t62.tif
t63.tif
t64.tif
t65.tif
   Loaded DIC-2/01_ST\SEG\man_seg006.tif, DIC-2/01\t006.tif
t67.tif
t68.tif
t69.tif
t70.tif
t71.tif
t72.tif
t73.tif
t74.tif
t75.tif
t76.tif
   Loaded DIC-2/01_ST\SEG\man

t613.tif
t614.tif
t615.tif
   Loaded DIC-2/01_ST\SEG\man_seg056.tif, DIC-2/01\t056.tif
t617.tif
t618.tif
t619.tif
t620.tif
t621.tif
t622.tif
t623.tif
t624.tif
t625.tif
t626.tif
   Loaded DIC-2/01_ST\SEG\man_seg057.tif, DIC-2/01\t057.tif
t628.tif
t629.tif
t630.tif
t631.tif
t632.tif
t633.tif
t634.tif
t635.tif
t636.tif
t637.tif
   Loaded DIC-2/01_ST\SEG\man_seg058.tif, DIC-2/01\t058.tif
t639.tif
t640.tif
t641.tif
t642.tif
t643.tif
t644.tif
t645.tif
t646.tif
t647.tif
t648.tif
   Loaded DIC-2/01_ST\SEG\man_seg059.tif, DIC-2/01\t059.tif
t650.tif
t651.tif
t652.tif
t653.tif
t654.tif
t655.tif
t656.tif
t657.tif
t658.tif
t659.tif
   Loaded DIC-2/01_ST\SEG\man_seg060.tif, DIC-2/01\t060.tif
t661.tif
t662.tif
t663.tif
t664.tif
t665.tif
t666.tif
t667.tif
t668.tif
t669.tif
t670.tif
   Loaded DIC-2/01_ST\SEG\man_seg061.tif, DIC-2/01\t061.tif
t672.tif
t673.tif
t674.tif
t675.tif
t676.tif
t677.tif
t678.tif
t679.tif
t680.tif
t681.tif
   Loaded DIC-2/01_ST\SEG\man_seg062.tif, DIC-2/01\t062.tif
t683.tif
t684

t1200.tif
t1201.tif
t1202.tif
t1203.tif
t1204.tif
t1205.tif
t1206.tif
t1207.tif
t1208.tif
t1209.tif
   Loaded DIC-2/02_ST\SEG\man_seg026.tif, DIC-2/02\t026.tif
t1211.tif
t1212.tif
t1213.tif
t1214.tif
t1215.tif
t1216.tif
t1217.tif
t1218.tif
t1219.tif
t1220.tif
   Loaded DIC-2/02_ST\SEG\man_seg027.tif, DIC-2/02\t027.tif
t1222.tif
t1223.tif
t1224.tif
t1225.tif
t1226.tif
t1227.tif
t1228.tif
t1229.tif
t1230.tif
t1231.tif
   Loaded DIC-2/02_ST\SEG\man_seg028.tif, DIC-2/02\t028.tif
t1233.tif
t1234.tif
t1235.tif
t1236.tif
t1237.tif
t1238.tif
t1239.tif
t1240.tif
t1241.tif
t1242.tif
   Loaded DIC-2/02_ST\SEG\man_seg029.tif, DIC-2/02\t029.tif
t1244.tif
t1245.tif
t1246.tif
t1247.tif
t1248.tif
t1249.tif
t1250.tif
t1251.tif
t1252.tif
t1253.tif
   Loaded DIC-2/02_ST\SEG\man_seg030.tif, DIC-2/02\t030.tif
t1255.tif
t1256.tif
t1257.tif
t1258.tif
t1259.tif
t1260.tif
t1261.tif
t1262.tif
t1263.tif
t1264.tif
   Loaded DIC-2/02_ST\SEG\man_seg031.tif, DIC-2/02\t031.tif
t1266.tif
t1267.tif
t1268.tif
t1269.tif


t1773.tif
t1774.tif
t1775.tif
t1776.tif
t1777.tif
t1778.tif
t1779.tif
t1780.tif
t1781.tif
   Loaded DIC-2/02_ST\SEG\man_seg078.tif, DIC-2/02\t078.tif
t1783.tif
t1784.tif
t1785.tif
t1786.tif
t1787.tif
t1788.tif
t1789.tif
t1790.tif
t1791.tif
t1792.tif
   Loaded DIC-2/02_ST\SEG\man_seg079.tif, DIC-2/02\t079.tif
t1794.tif
t1795.tif
t1796.tif
t1797.tif
t1798.tif
t1799.tif
t1800.tif
t1801.tif
t1802.tif
t1803.tif
   Loaded DIC-2/02_ST\SEG\man_seg080.tif, DIC-2/02\t080.tif
t1805.tif
t1806.tif
t1807.tif
t1808.tif
t1809.tif
t1810.tif
t1811.tif
t1812.tif
t1813.tif
t1814.tif
   Loaded DIC-2/02_ST\SEG\man_seg081.tif, DIC-2/02\t081.tif
t1816.tif
t1817.tif
t1818.tif
t1819.tif
t1820.tif
t1821.tif
t1822.tif
t1823.tif
t1824.tif
t1825.tif
   Loaded DIC-2/02_ST\SEG\man_seg082.tif, DIC-2/02\t082.tif
t1827.tif
t1828.tif
t1829.tif
t1830.tif
t1831.tif
t1832.tif
t1833.tif
t1834.tif
t1835.tif
t1836.tif
   Loaded DIC-2/02_ST\SEG\man_seg083.tif, DIC-2/02\t083.tif
t1838.tif
t1839.tif
t1840.tif
t1841.tif
t1842.tif


In [ ]:
# def create_preprocessed_cache(dataset, new_dir_name=None):
    
#     if new_dir_name == None:
#         new_dir_name = dataset+"_cache"
    
#     try:
#         os.mkdir(new_dir_name)
#         os.mkdir(os.path.join(new_dir_name, "originals"))
#         os.mkdir(os.path.join(new_dir_name, "clahes"))
#         os.mkdir(os.path.join(new_dir_name, "masks"))
#         os.mkdir(os.path.join(new_dir_name, "markers"))
#         os.mkdir(os.path.join(new_dir_name, "weight_maps"))
#     except:
#         print("One of the directories to be created already exists. Clear and try again.")
#         return
    
#     data = []
#     paths = [dataset + '/01', dataset + '/02']
#     count = 0
    
#     for path in paths:
#         mask_path = path + '_ST'
#         mask_path = os.path.join(mask_path, "SEG")
#         for f in os.listdir(mask_path):
#             if not f.endswith(".tif"):
#                 continue
#             image = cv2.imread(os.path.join(path, f.replace('man_seg', 't')), cv2.IMREAD_GRAYSCALE)
#             clahe = equalize_clahe(image).astype(np.float32)
#             mask = cv2.imread(os.path.join(mask_path, f), cv2.IMREAD_UNCHANGED)
#             print("   Loaded " + os.path.join(mask_path, f) + ", " + os.path.join(path, f.replace('man_seg', 't')))
            
#             # Generate the Cell Mask and Markers from the Mask
#             cell_mask = (mask > 0).astype(np.uint8)
#             markers = (get_markers(mask) > 0).astype(np.uint8)
#             weight_map = get_weight_map(markers)
            
#             index = 't' + str(count) + ".tif"
#             count += 1
            
# #             np.save(os.path.join(new_dir_name, "originals", index ), image)
# #             np.save(os.path.join(new_dir_name, "clahes", index ), clahe)
# #             np.save(os.path.join(new_dir_name, "masks", index ), cell_mask)
# #             np.save(os.path.join(new_dir_name, "markers", index ), markers)
# #             np.save(os.path.join(new_dir_name, "weight_maps", index ), weight_map)
            
#             cv2.imwrite(os.path.join(new_dir_name, "originals", index ), image)
#             cv2.imwrite(os.path.join(new_dir_name, "clahes", index ), clahe)
#             cv2.imwrite(os.path.join(new_dir_name, "masks", index ), cell_mask)
#             cv2.imwrite(os.path.join(new_dir_name, "markers", index ), markers)
#             cv2.imwrite(os.path.join(new_dir_name, "weight_maps", index ), weight_map)

# create_preprocessed_cache('DIC-2', new_dir_name=None)

In [ ]:
# cell_masks = get_cell_masks()
# origs = get_origs()
# cell_markers = get_cell_markers(cell_masks, 24)
# weight_maps = get_weight_maps(cell_markers, False)

# a = origs[1]
# b = get_clahes(origs)[1]
# c = cell_masks[1]
# d = cell_markers[1]

# h = weight_maps[1]


# pimg(a)

# pimg(b)

# pimg(c)

# pimg(d)

# pimg(h)

In [ ]:
# cell_masks = get_cell_masks()
# origs = get_origs()
# cell_markers = get_cell_markers(cell_masks, 8)
# weight_maps = get_weight_maps(cell_markers, False)

# a = origs[0]
# b = get_clahes(origs)[0]
# c = cell_masks[0]
# d = cell_markers[0]

# h = weight_maps[0]


# pimg(a)

# pimg(b)

# pimg(c)

# pimg(d)

# pimg(h)

In [ ]:
# cell_masks = get_cell_masks()
# origs = get_origs()
# cell_markers = get_cell_markers(cell_masks, 2)
# weight_maps = get_weight_maps(cell_markers, False)

# a = origs[2]
# b = get_clahes(origs)[2]
# c = cell_masks[2]
# d = cell_markers[2]

# h = weight_maps[2]


# pimg(a)

# pimg(b)

# pimg(c)

# pimg(d)

# pimg(h)